# Import Libraries

In [ ]:
import os
import cv2
import shutil
import random

import numpy as np
import matplotlib.pyplot as plt

from ultralytics import YOLO

# Load Data

In [ ]:
data_dir = "/mnt/d/Datasets/yolo-traffic-signs/ts/ts/"
output_dir = "/mnt/d/Datasets/yolo-traffic-signs/yolo-data"

In [ ]:
os.makedirs(output_dir, exist_ok=True)

In [ ]:
train_images_dir = os.path.join(output_dir, "train/images")
train_labels_dir = os.path.join(output_dir, "train/labels")
val_images_dir = os.path.join(output_dir, "val/images")
val_labels_dir = os.path.join(output_dir, "val/labels")

os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

In [ ]:
train_ratio = 0.8

In [ ]:
files = os.listdir(data_dir)
jpg_files = [f for f in files if f.endswith(".jpg")]
txt_files = [f for f in files if f.endswith(".txt")]

In [ ]:
combined_files = list(zip(jpg_files, txt_files))
random.shuffle(combined_files)
train_size = int(len(combined_files) * train_ratio)

In [ ]:
train_files = combined_files[:train_size]
val_files = combined_files[train_size:]

In [ ]:
def copy_files(file_pairs, img_dest, lbl_dest):
    for img_file, txt_file in file_pairs:
        shutil.copy(os.path.join(data_dir, img_file), os.path.join(img_dest, img_file))
        shutil.copy(os.path.join(data_dir, txt_file), os.path.join(lbl_dest, txt_file))

In [ ]:
copy_files(train_files, train_images_dir, train_labels_dir)

In [ ]:
copy_files(val_files, val_images_dir, val_labels_dir)

In [ ]:
data_yaml = f"""
names:
  0: prohibitory
  1: danger
  2: mandatory
  3: other
nc: 4
train: {os.path.abspath(train_images_dir)}
val: {os.path.abspath(val_images_dir)}
"""

In [ ]:
with open(os.path.join(output_dir, "data.yaml"), "w") as f:
    f.write(data_yaml)

f.close()

# Visualization

In [ ]:
def plot_images_with_bboxes(samples, images_dir, labels_dir, class_names):
    fig, axes = plt.subplots(nrows=len(samples), ncols=1, figsize=(8, 16))
    axes = axes.flatten()
    
    for idx, sample in enumerate(samples):
        img_path = os.path.join(images_dir, sample[0])
        label_path = os.path.join(labels_dir, sample[1])
        image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
        
        if os.path.exists(label_path):
            with open(label_path, 'r') as file:
                for line in file:
                    class_id, x, y, w, h = map(float, line.split())
                    h_img, w_img, _ = image.shape
                    x, y, w, h = x * w_img, y * h_img, w * w_img, h * h_img
                    x_min, y_min = int(x - w / 2), int(y - h / 2)
                    x_max, y_max = int(x + w / 2), int(y + h / 2)
                    
                    cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 0, 255), 2)
                    cv2.putText(image, class_names[int(class_id)], (x_min, max(y_min - 10, 10)), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        axes[idx].imshow(image)
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
samples = random.sample(train_files, 4)

In [ ]:
class_names = ["prohibitory", "danger", "mandatory", "other"]

In [ ]:
plot_images_with_bboxes(samples, train_images_dir, train_labels_dir, class_names)

# Train

In [ ]:
model = YOLO('yolov8x.pt')

In [ ]:
results = model.train(
    data = os.path.join(output_dir, "data.yaml"),
    epochs = 50,
    seed = 42,
    batch = 16,
)

# Results

In [ ]:
metrics = model.val(
    data = os.path.join(output_dir, "data.yaml"),
)

In [ ]:
df = pd.read_csv("/kaggle/working/runs/detect/train2/results.csv")
df.columns = df.columns.str.strip()

In [ ]:
metrics_list = [
    ('train/box_loss', 'Train Box Loss'),
    ('train/cls_loss', 'Train Class Loss'),
    ('train/dfl_loss', 'Train DFL Loss'),
    ('metrics/precision(B)', 'Metrics Precision (B)'),
    ('metrics/recall(B)', 'Metrics Recall (B)'),
    ('metrics/mAP50(B)', 'Metrics mAP50 (B)'),
    ('metrics/mAP50-95(B)', 'Metrics mAP50-95 (B)'),
    ('val/box_loss', 'Validation Box Loss'),
    ('val/cls_loss', 'Validation Class Loss'),
    ('val/dfl_loss', 'Validation DFL Loss'),
]

fig, axs = plt.subplots(nrows=5, ncols=2, figsize=(15, 15))
for ax, (col, title) in zip(axs.flatten(), metrics_list):
    sns.lineplot(x='epoch', y=col, data=df, ax=ax).set(title=title)

plt.suptitle('Training Metrics and Loss', fontsize=24)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

In [ ]:
conf_matrix = plt.imread('/kaggle/working/runs/detect/train/confusion_matrix_normalized.png')

plt.figure(figsize=(8, 8))
plt.imshow(conf_matrix)
plt.axis('off')
plt.show()

In [ ]:
p_curve = plt.imread('/kaggle/working/runs/detect/train/P_curve.png')

plt.figure(figsize=(8, 8))
plt.imshow(p_curve)
plt.axis('off')
plt.show()

In [ ]:
r_curve = plt.imread('/kaggle/working/runs/detect/train/R_curve.png')

plt.figure(figsize=(8, 8))
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
pr_curve = plt.imread('/kaggle/working/runs/detect/train/PR_curve.png')

plt.figure(figsize=(8, 8))
plt.imshow(img)
plt.axis('off')
plt.show()

# Test

In [ ]:
!yolo task=detect mode=predict model=/kaggle/working/runs/detect/train2/weights/best.pt \
source='./yolo-data/train/images/00028.jpg'

In [ ]:
test_image = plt.imread('/kaggle/working/runs/detect/predict/00028.jpg')

plt.figure(figsize=(8, 8))
plt.imshow(test_image)
plt.axis('off')
plt.show()

In [ ]:
!yolo task=detect mode=predict model=/kaggle/working/runs/detect/train/weights/best.pt \
source='/mnt/d/Datasets/yolo-traffic-signs/traffic-sign-to-test.mp4'